In [1]:
from sklearn import datasets
from credal_joint import CredalJoint
from graphs import Node
import numpy as np
from itertools import product
from new_tree import DecisionTree
from scipy.optimize import minimize, Bounds
from scipy.stats import entropy
from sklearn.metrics import mutual_info_score

In [2]:
iris = datasets.load_iris()

In [3]:
X = iris['data']

In [4]:
y = iris['target'].reshape(-1, 1)

In [54]:
def discretize(X:np.ndarray, n_cat:int, min_ar:np.ndarray=None, max_ar:np.ndarray=None) -> np.ndarray:
    """
    Digitizes the given data into a given number of categories.

        Parameters:
            X (np.ndarray): The data to digitize.
            n_cat (int): The number of categories to digitize the data into.
            min (float): The minimum value of the data.
            max (float): The maximum value of the data.

        Returns:
            np.ndarray: The digitized data.
    """
    new_X = np.zeros(X.shape)
    if min_ar is None:
        min_ar = np.min(X, axis=0)
    if max_ar is None:
        max_ar = np.max(X, axis=0)
    for i in range(X.shape[1]):
        new_X[:,i] = np.digitize(X[:,i], np.linspace(min_ar[i], max_ar[i], n_cat), False)
    return new_X, min_ar, max_ar

In [55]:
X_cat_train, min_ar, max_ar = digitize(X[:int(X.shape[0] / 2)], 3)
X_cat_test, _, _ = digitize(X[int(X.shape[0] / 2):], 3, min_ar, max_ar)